# Results from Thomas et al. (2024)
https://github.com/Shef-AIRE/llms_post-ocr_correction/tree/main/results

In [1]:
# git clone Shef-AIRE/llms_post-ocr_correction
import pandas as pd
url = "https://raw.githubusercontent.com/Shef-AIRE/llms_post-ocr_correction/refs/heads/main/results/llama-2-13b.csv"
data=pd.read_csv(url,sep=',',index_col=0)


In [2]:
def assign_CER_interval(df):
    
    if df.old_CER >= 0.1:
        interval = '>=0.1'
    elif df.old_CER < 0.1 and df.old_CER != 0:
        interval = '<0.1&!=0'
    elif df.old_CER == 0:
        interval = '0'

    return interval


data['CER_interval'] = data.apply(assign_CER_interval,axis=1)
data['len']=data['OCR Text'].map(len)
print(f'Average CER reduction across long sentences > 400 chars: {data.query('len > 400').CER_reduction.mean()}')
print(f'Average CER reduction across sentences of length under 400 chars: {data.query('len < 400').CER_reduction.mean()}')
print(f'Counts of CER interval types in long sentences in test:\n {data.query('len > 400').CER_interval.value_counts()}')
print(f'Average input length in test: {data.len.mean()}')

Average CER reduction across long sentences > 400 chars: -189.97296467157798
Average CER reduction across sentences of length under 400 chars: 59.50484114119872
Counts of CER interval types in long sentences in test:
 CER_interval
<0.1&!=0    31
>=0.1        7
Name: count, dtype: int64
Average input length in test: 124.94448424068769


# Calculate Metrics for German Post-OCR Correction

In [ ]:
import pandas as pd
from jiwer import wer, cer
# from german_lemmatizer import lemmatize
import unicodedata
import re, os

In [ ]:
def filter_unicode(text):
    # Filter out non-printable characters using Unicode character categories
    # Keep only characters that are of type: Letter, Number, Punctuation, Symbol, or Space Separator
    return ''.join(
        char for char in text
        if unicodedata.category(char).startswith(('L', 'N', 'P', 'S', 'Z'))
    )

In [ ]:
def post_process(data, column):
    data[column] = data[column].apply(lambda text: re.sub('^[\n\s]','', text))
    data[column] = data[column].apply(lambda text: re.sub('\n','', text))
    data[column] = data[column].apply(lambda text: re.sub('\s{2,}',' ', text))
    data[column] = data[column].apply(lambda text: filter_unicode(text))

    return data

In [ ]:
def assign_CER_interval(df):

    metric = df.old_CER
    
    if metric >= 0.1:
        interval = '>=0.1'
    elif metric < 0.1 and metric != 0:
        interval = '<0.1&!=0'
    elif metric == 0:
        interval = '0'

    return interval

In [ ]:
columns = ['OCR Text', 'Ground Truth', 'Model Correction']

def calc_metrics(path):
    
    df=pd.read_csv(path)
    
    df = post_process(df, 'OCR Text')
    df = post_process(df, 'Ground Truth')
    df = post_process(df, 'Model Correction')

    df['old_CER'] = df.apply(lambda row: cer(row['Ground Truth'], row['OCR Text']), axis=1)
    df['old_WER'] = df.apply(lambda row: wer(row['Ground Truth'], row['OCR Text']), axis=1)

    df['new_CER'] = df.apply(lambda row: cer(row['Ground Truth'], row['Model Correction']), axis=1)
    df['new_WER'] = df.apply(lambda row: wer(row['Ground Truth'], row['Model Correction']), axis=1)

    df['CER_interval'] = df.apply(assign_CER_interval,axis=1)

    df['CER_reduction'] = ((df['old_CER'] - df['new_CER']) / df['old_CER']) * 100

    df['WER_reduction'] = ((df['old_WER'] - df['new_WER']) / df['old_WER']) * 100

    return df

In [ ]:
from scipy import stats
import numpy as np
dir="results"
corrs=[]
results=[]
def report_metrics(dir):
    global corrs
    global results
    for file in os.listdir(dir):
        
        if file.endswith('.csv'):
            df = calc_metrics(os.path.join(dir,file))
            df.to_csv('results_corrected/'+file)
            df=df.query('Publication in ["Diabetiker_journal"]')

            print(f'results of: {file.split(".")[0]}\n')
            # print(f"Portion of improvement: {len(df.query('CER_reduction == 0'))/len(df)}")
            # print(df['CER_reduction'].isna().sum()/len(df))

            df['CER_reduction']=df['CER_reduction'].fillna(0)
            df['WER_reduction']=df['WER_reduction'].fillna(0)

            df['CER_reduction'] = df['CER_reduction'].replace(-np.inf, -1)
            df['WER_reduction'] = df['WER_reduction'].replace(-np.inf, -1)
            # print(len(df.query('CER_reduction == -inf')))

            same_error = len(df.query('CER_reduction == 0 and CER_interval not in ["0"]'))/len(df)
            same_perfect = len(df.query('CER_reduction == 0 and CER_interval in ["0"]'))/len(df.query('CER_interval in ["0"]'))
            print(f"Portion of the same error in the test set with non-zero interval: {same_error}")
            print(f"Portion of preserved zero CER in zero interval: {same_perfect}")

            # print(f'old cer: {df.old_CER.mean()}\nnew cer: {df.new_CER.mean()}\n')

            results+=[(file.split(".")[0],
                   same_error, same_perfect)]
            #############
            # print(df['old_WER'].corr(df['old_CER']))
            # print(df['new_WER'].corr(df['new_CER']))
            # corrs+=[df['old_WER'].corr(df['old_CER'])]
            #################
            # print(((df.old_WER.mean() - df.new_WER.mean()) / df.old_WER.mean()) * 100)

            # case=df.query('CER_interval == ">=0.1"')
            # print(f"Portion of improvement in difficult errors, cer >=0.1: {len(case.query('CER_reduction > 0'))/len(case)}")
            
            # major_CER=len(case.query('CER_reduction > 0'))/len(case)

            # case=df.query('CER_interval in ["<0.1&!=0"]')
            # print(f"Portion of improvement in mid-difficult errors, cer <0.1&!=0: {len(case.query('CER_reduction > 0'))/len(case)}")

            # minor_CER=len(case.query('CER_reduction > 0'))/len(case)

            # case=df.query('CER_interval in [">=0.1", "<0.1&!=0"]')
            # print(f"Portion of improvement in all errors, cer != 0: {len(case.query('CER_reduction > 0'))/len(case)}")

            # both_CER=len(case.query('CER_reduction > 0'))/len(case)
            
            # case=df.query('CER_interval in ["0"]')
            # print(f"Portion of deterioration in zero error case: {len(case.query('CER_reduction < 0'))/len(case)}")

            # print(f'old wer: {df.old_WER.mean()}\nnew wer: {df.new_WER.mean()}\n')

            # case=df.query('CER_interval == ">=0.1"')
            # print(f"Portion of improvement in difficult errors WER , cer >=0.1: {len(case.query('WER_reduction > 0'))/len(case)}")
            
            # major_WER=len(case.query('WER_reduction > 0'))/len(case)

            # case=df.query('CER_interval in ["<0.1&!=0"]')
            # print(f"Portion of improvement in mid-difficult errors WER, cer <0.1&!=0: {len(case.query('WER_reduction > 0'))/len(case)}")
            
            # minor_WER=len(case.query('WER_reduction > 0'))/len(case)

            # case=df.query('CER_interval in [">=0.1", "<0.1&!=0"]')
            # print(f"Portion of improvement in all errors WER, cer != 0: {len(case.query('WER_reduction > 0'))/len(case)}")

            # both_WER=len(case.query('WER_reduction > 0'))/len(case)

            # case=df.query('CER_interval in ["0"]')
            # print(f"Portion of deterioration in zero error case WER: {len(case.query('WER_reduction < 0'))/len(case)}")

            # #'model',
            # results+=[(file.split(".")[0],
            #        minor_CER, minor_WER, major_CER, major_WER, both_CER, both_WER)]

            print('\n***')
report_metrics(dir)

# Zero- & Few-Shot Inference

In [5]:
prompt = """###Anweisungen: Korrigieren Sie OCR-Fehler im folgenden Text.
OCR-Fehler umfassen Leerzeichen-Fehler, Umlaut-Fehler, Eszett-Verwechslungen, Zeichensubstitutionsfehler, Ligaturen-Fehler, Trennungsprobleme, Groß- und Kleinschreibungsfehler, Interpunktionsfehler.
###Beispiele:
[INST]
OCR: und tonzentralen bringen Land und
Leute naher und lassen die Schie-
nen-Kreuzfahrt jedes Mal zum ein-
maligen Erlobnis werden.
IIIIIIIIIII /
Die moderne, junge
Die MTS CASTALIA ist ein elegantes Kreuz- • 7,5
[/INST]
Korrektur: und Tonzentralen bringen Land und
Leute näher und lassen die Schie¬
nen-Kreuzfahrt jedes Mal zum ein¬
maligen Erlebnis werden.
Die moderne, junge und schnelle MTS CASTALIA - ein schwimmendes Hotel
[INST]
OCR: kann auch auf eine g e n a u e A n g a b e d e r A r t
des verwendeten Zuckeraustausch-
stoffes nicht verzichtet werden. auch versteht
es sich von selbst, daß j e d w e d e r K o n s e r v i e-
[/INST]
Korrektur: kann auch auf eine genaue Angabe der Art
des verwendeten Zuckeraustausch¬
stoffes nicht verzichtet werden. Auch versteht
es sich von selbst, daß jedweder Konservie¬
[INST]
OCR: KURHOTEL
cuiheim
Gesamtleitung:
hanns Kilian
W I G G E R
GARMISCH-PARTENKIRCHEN
das derzeit fuhrende Haus in unvergleichlich
ruhiger und schoner sonnentage mit klinisch
[/INST]
Korrektur:
"""


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_name = "jphme/em_german_13b_v01"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    temperature=0.7,
    do_sample=True,
    max_new_tokens=1024,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id
)

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


###Anweisungen: Korrigieren Sie OCR-Fehler im folgenden Text.
OCR-Fehler umfassen Leerzeichen-Fehler, Umlaut-Fehler, Eszett-Verwechslungen, Zeichensubstitutionsfehler, Ligaturen-Fehler, Trennungsprobleme, Groß- und Kleinschreibungsfehler, Interpunktionsfehler.
###Beispiele:
[INST]
OCR: und tonzentralen bringen Land und
Leute naher und lassen die Schie-
nen-Kreuzfahrt jedes Mal zum ein-
maligen Erlobnis werden.
IIIIIIIIIII /
Die moderne, junge
Die MTS CASTALIA ist ein elegantes Kreuz- • 7,5
[/INST]
Korrektur: und Tonzentralen bringen Land und
Leute näher und lassen die Schie¬
nen-Kreuzfahrt jedes Mal zum ein¬
maligen Erlebnis werden.
Die moderne, junge und schnelle MTS CASTALIA - ein schwimmendes Hotel
[INST]
OCR: kann auch auf eine g e n a u e A n g a b e d e r A r t
des verwendeten Zuckeraustausch-
stoffes nicht verzichtet werden. auch versteht
es sich von selbst, daß j e d w e d e r K o n s e r v i e-
[/INST]
Korrektur: kann auch auf eine genaue Angabe der Art
des verwendeten Zuckera

## ocronos

In [9]:
from vllm import LLM, SamplingParams

# initialize
# sampling_params = SamplingParams(temperature=0.9, top_p=.95, max_tokens=4000, presence_penalty = 0, stop=["#END#"])
# llm = LLM(model="PleIAs/OCRonos", gpu_memory_utilization=0.80)
# perform the inference
input = """
KURHOTEL
cuiheim
Gesamtleitung:
hanns Kilian
W I G G E R
GARMISCH-PARTENKIRCHEN
das derzeit fuhrende Haus in unvergleichlich
ruhiger und schoner sonnentage mit klinisch"""
input="""
Verbrauchers ist.
(Anschr d. Verf.: Köln-Riehl, Theodor-Schwann-Str. 4)
für Diabetiker sehr zu empfehlen
Reimdiät
^■^■■■■H nach Dr. Grande!
"""
input="""
von Diabetikern besonders geschätzt.
Schulte-Maure!
Kornbrennerei seit 1848, Castrop-Rauxel
...zpoaäfes Ce Zhre diätetischen Vabriegsmitlel 2
"""
prompt_ = "### Text ###\n" + input + "\n\n### Correction ###\n"
outputs = llm.generate(prompt_, sampling_params, use_tqdm = False)

# print outputs
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


Prompt: '### Text ###\n\nvon Diabetikern besonders geschätzt.\nSchulte-Maure!\nKornbrennerei seit 1848, Castrop-Rauxel\n...zpoaäfes Ce Zhre diätetischen Vabriegsmitlel 2\n\n\n### Correction ###\n', Generated text: 'von Diabetikern besonders geschätzt. Schulte-Maure! Kornbrennerei seit 1848, Castrop-Rauxel...zpoaäfes Ce Zhre diätetischen Vabriegsmittel 2 '


Prompt: '### Text ###\n\nKURHOTEL\ncuiheim\nGesamtleitung:\nhanns Kilian\nW I G G E R\nGARMISCH-PARTENKIRCHEN\ndas derzeit fuhrende Haus in unvergleichlich\nruhiger und schoner sonnentage mit klinisch\n\n### Correction ###\n', 

Generated text: 'KURHOTEL\ncuiheim\nGesamtleitung:\nhanns Kilian\nWIGGER\nGARMISCH-PARTENKIRCHEN\ndas derzeit fuhrende Haus in unvergleichlich\nruhiger und schöner sonnentage mit klinisch\n\nKürzest. Werbe. Auflage. \nEinzigartige Konzep. Di. Hoflstar. Des. Steinfelderhöhe. Für. Lebendige. Dein. Naturheil. Wunder. Hochaußichte. A. Bergland. Di. Erholung. Gar. Wohlsein. Höhenluft. Gesundheit. Di. Cuiheim. Hausmanin. Er. Kurgast. Ausstiegshöhe. Amtliche. Gebäude. Beid. Bahnstatione. Steinfelde un. Re. Neuenfett. Ihr. Ohrsan. Wunde. Zähne. Ohren. Knie. Hüfte. Nieren. Herz. Haut. I. Geschenke. Vorsehung. Dein. Zuflucht. Jede. Zeiten. Wunde. I. Strahlenglan. Dein. Linderung. Versuch. Dein. Kräfte. Der. Erschöpfung. I. Gelingens. Reich. Dein. Energi. Inner un Äusser. Zufluss. Ei. Regenbogen. Dei. Ausblick. Di. Morgenröte. Dei. Niedrigkewi de. Sonne. Mitternach. Dei. Abendrote. Jede. Tag . Ei neue. Wunder. Dei. Natur. I de beste. Gliederde de. Berge. Ei. Felsensäule. Dei. Quellen. Ei. Fliege ohne. Flügel. Ei. Querblic de. Auge. I de prachtvolle. Lauskamme un. Tannengran. Der. Fels un de. Eise. Jede. Spalt. Ei. Blumenspi. Dei. Natur. Jede. Ma. Ei neue. Wunder. I. Erdbode. Eine. Hüll vo runde. Felse. Ei. Lus un. Wiese. Stal de. Vieh un de. Mensch. Jede. Gipfel. Ei. Kreuz. Jede. Winkels. Ei. Minut. Vergeht. Jede. Secon. Ei neue. Wunder. G. V?H-. T??. GEB?UTE. FREU\'. EHR. GERECHTIGKEIT. Jede. Zelle. Ei. Quarz. I de tiefste. Schiessen. Ei. Kristall. Di. Lüfte un di. Sonn . Ei. Zauberwir. Erhabenhei. Jede. Frische. Ei neue. Wunder. Jede. Tröste. Ei neue. Blü. Di. Naturheil. Wunder. I de. Bergen. Ei. Morgenblic de. Alte. Römis. Veld. Marsc. Di. Frühling. Wunder. I de  1. Jahrhundert. Di. Naturheil. Wunder. I de. Bergen. Tausende vo alte un neue. Mensch. Glaube auc jetz a. Naturheil. Wunder. Tausende ha de. Glauben wi jene vo. Alzheim. H. Ehrenfriedber au. Kaiserslautern i. Jahr 1510. Ka ei. Prinzi ausgewählt. Wo sic j. Jahrhundert. Di. Kurgaste eingefundene. Spaziergänge. De. Ohrsan. Wunde unte ander gehören. De. Eifelturm. Hammer un. Fichtenfeldkappe. De. Saargipfel. Schau un. Hohenbuh i. Saargebie. De. Donnersber u. Steinbrücke. Ei. Tagauszhaltun vo 7—9. Stunde. Wen. Noc ei größer. Kraf nötig. O das. Weid i. Kirchliebhü. Buchholzhü un. Rohrbach. Weiterspaziergänge vo 2—2 1/2. Stunde. Weid de. Rote un. Weiße. Turm i. Gharlierberg. De. Kaltenbrunnerhöhe. Silbertuh un. Michelhamm. Bahnspfahre gege klein. Entgel vo  80. Pfennig. Ei. Kursuss vo 3—4. Persone (4. Tag zwische. Mahlzeiten. Nachmitta un. Abend. Erfahrun etwa. Naturheil. Wunder. I de. Bergen. Prei 1. Fc 1. Pro. Iw 1. Pro. Kap. Uebersendun vo. Anfrag zu. Kreditorenordnun i. Anbetr. Verstorbene. Erwiderun (De. Nekrolog betrifft. Frw. Amalie. Kuhn. Gestorb 22. Februar. I de . Rheni fche. Westfali fche. Zeitun unte de. Rubri. Ein. Seemanns. To de. Nekrolo aufgenomme wo den. Se . Namensbe. Amalie. Kuhnin. Jahr 183 i diese. Stad gestorbe ist habe da. Künstlerfrüh. Ihr. Töchte da. Sterb. Bild gemalt. De. Künstle dankte ih fü dies. Geschen . Großmutter! E ha da. Künstlerfrüh ihr gern gemalt. Ihr. Tochte dankt ih fü dies. Geschenk „Großmutter! ... Da. Klei zeige a. Schloss. Bensheim i de. Königsstau. \nMai. IqS?«. Lauxenberge. —. Zu. Festläute de i de. Bitterzit zu. Vaterland. Un. Kirch-Verfassung vo 5. Iahrl. Hunder. Jah. Fei. Jubilu veranstaltende. Glieder de evangelische. Gottes. Aelser. Bürger. Gsmeil un. Schützenverein. Lauxenberge wir hie mi. Gnad. De. Kaiserl. Königliche. Allerhöchsste. Ent schliessun zu. Eigenthum „dem sgenannte. Gliedern un. Mitglieder dieses. Vereine unte de. Schutze. Dei. Bürgermeister. Vogtliche. Einwilligun un de. Bezirksoberam. Behörd gelangte. Anlage betrefende de unter 1 de. Statut bezeichnete. Festvortra z de öffentliche. Vortrag i gänzliche. Wortlaut unte ander folgt. Liede be diese. Festvortra z singen. Nr. O . Deutliche l?hne durc de. Wah. Römisch-katholisch. Kirch. Lieder (Gr. Liedbhnn. I. Abtheilu 184 un 21. S.. Di. Auseinandersetzung welch de vo. Dominicus de. Bader. Z. St. Wendhe zu. Begin de 13. Jahrhundert i de. Berline. Kirch. St. Hrn gelesene 1. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 2. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 3. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 4. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 5. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 6. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 7. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 8. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 9. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 10. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 11. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 12. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 13. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 14. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 15. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 16. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 17. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 18. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 19. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 20. Hurrianisch. Di. Auseinandersetzung welch de vo. Dominicus de. Bader i de. Berliene. Kirch. St. Hrn zu. Begin de 13. Jahrhundert gelesene 21. Hurrianisch. De. Hymn au de. Taufsliede. L« Änhn 8 un. De. Taufspesiel. Nr 2. Di. Auseinandersetzung welch di. Summlieddichte zwische 300 un 200 un 100 vo. S. Erna. Juli. Gauw. Mlndthurn un. Ne. R. Un. C. S. Nr i vermitteln . Lauxenberge 5. Oktobe 1845. —. Vo. Hm de. Herr. Bürgermeister in. Lauxenberg habe sic de. Bischo de. Römisch-katholische. Kirch. Diöze. Speyer. Se. Excellenz. Herr. Dr. Ber (S)entin ehemalige. Curato de. Stifte. S. Saffran i de. Pfarre. Lauxenberg. Vorstehe un. Administrato de. Stifte. S. Saffran ernenne z lasse g seßt. Wie de 18. Okrobe 1845. —. Vo. Hm de. Herr. Bürgermeister i. Lauxenberg habe sic de. Herr. Heinric. Gnsel. Man. Ann. Cun. Col. Erbschmidtha di. Königl. Land-Ziöse un. Oberamts-Kammer-Rath i. Speyer zu. Woh de. Gemeind z  übegeben un bestimme nac vollendete. Jahrelange daselbs di. Leistun eine. Gemeindefrau. Di. Heiligegeistin. Ann. Km au de. Golse i. Speyer ehemalige. Burger-Magd i. Lauxenberg de sic währen de. Pockenge fäll 181 au besondere. Befeh de. Obrist-Sanitätts-Präsidente i. Speyer welch de gan. K. Jüche durc diese. Seuche beute un di. Provin nac de. Sparsamkei ihres. Lebens. Al. Witw i. Lauxenberge weithin weithin weithin weithin bekannt un geliebt. Frau erworbe hat diefes. Ehre z se. Eine ode beide de. Stifter de. S. Saffran durc ihr. Tod z. Thei gewordenen. Anna-Gchmitt. Summlieder z de. Vorlesen. Vierte diese. Lieder sei de. Hymn „Die. Taufe" welch sei de „Niederwerfung. Christi de. T. Pau. Gerhardt welch ei vo. E eingelegte un au diese. Jah z lebendige. Wiedc erschienene. Liede is abe dies. Text un diese. Melod gewi au 18. Jahrhunder vo de. Musi nac de. Liede „War cum. Trnba un „Lobet de. Her un da. T. F. E.. Gellert. Au diese. Jahr hinau d durc de. Tod de. Künstler de gesammte. Musi verloren gebliebe ist. Da d. Niede de. Haltun diese zwey. Hochsäürge. Liede welch scho i 24 un i 30. Ma. Weimarsch vonSt.-Vith un durc de. Se. Excellenz. Herr. Bischö de. Diöze. Spires imge. Jahr vo erste. Male i. Lauxenberg gewöhnlic z. Aufführun gekommen al a 6. Oktobe d. J be dritte. Male wiederhol will wiede aufgeführt werden. Vo de. Texte de 2. Kirchenlieds im 7 un 8. Strophe werde. I de 7. Strophe di. Auseinandersetzung welch de. H. Paulu. S. Saffran i de „Chronik seine. Stifte. S. Saffran i. Jahr 145 a de. Römisch-katholische. Kirch z. Lauxenberg gege de „Deutsche. Orden abgeführt abgeführt. I de 8. Strophe de. Hymn „De. Taufe wir da. Ereignis welch de i 13. Jahrhundert i. Lauxenberg gestorben. H. Dominik. Bader un de. H. Hrn. Paulu. Saffran betreffen welche au de. Ordnung de. H. Dominik. Bader . Römisch-katholische. Kirch i de. Pfarre. Lauxenberg. Hau. Kirche eingetretene. Herr. Paulu. Saffran z diese. Beruff gebrach hatte worübe de. Se. Excellenz. Bischo z seine. Zei sic de. Römisch-katholische. Kirch de. Gemeind. Lauxenberg a dere. Hau. Kirch z. Nutzun diese wertvolle 2. Kirchenlieder nicht erschwer habe wird. Di. Aufführun de. Musi sol u. U ge. Mitta begine un nac vollständige. Uebertragun de. Texte durc de. Se. Excellen. Bischo. Hrn au dies. Tage folgend. Gottes. Dienstes durc de. Herr. Gnsel. Kindertan abgeschlossen abgeschlossen werden. Wie de '

Prompt: '### Text ###\n\nVerbrauchers ist.\n(Anschr d. Verf.: Köln-Riehl, Theodor-Schwann-Str. 4)\nfür Diabetiker sehr zu empfehlen\nReimdiät\n^■^■■■■H nach Dr. Grande!\n\n\n### 

Correction ###\n', 
Generated text: 'Verbrauchers ist.\n(Anschrift d. Verf.: Köln-Riehl, Theodor-Schwann-Str. 4)\nfür Diabetiker sehr zu empfehlen\nReimdiät\nnach Dr. Grande! '

Prompt: '### Text ###\n\nvon Diabetikern besonders geschätzt.\nSchulte-Maure!\nKornbrennerei seit 1848, Castrop-Rauxel\n...zpoaäfes Ce Zhre diätetischen Vabriegsmitlel 2\n\n\n### 

Correction ###\n', Generated text: 'von Diabetikern besonders geschätzt. Schulte-Maure! Kornbrennerei seit 1848, Castrop-Rauxel...zpoaäfes Ce Zhre diätetischen Vabriegsmittel 2 '